In [2]:
import nltk
# nltk.download("vader_lexicon") # only need to run this once
from nltk.sentiment.vader import SentimentIntensityAnalyzer
vader = SentimentIntensityAnalyzer()
import pandas as pd

In [3]:
# simple example
data = pd.read_csv('../data/example.csv')

In [4]:
print(data['text'][0])
print(data['text'][1])
print(data['text'][2])
print(data['text'][3])
  

This is a great product - I really love it. I received my order quickly with no problems. It looks amazing. But the manual is not good.
This is a great product - I really love it. I received my order quickly with no problems. It looks amazing
What a horrible product! It's totally useless. There is no point in buying this even if you like the brand
This is not a terrible product. It really isn't that bad. It also not great.


In [5]:
sentiments = [vader.polarity_scores(document) for document in data['text']]
sentiments

[{'neg': 0.21, 'neu': 0.544, 'pos': 0.246, 'compound': 0.2818},
 {'neg': 0.167, 'neu': 0.41, 'pos': 0.423, 'compound': 0.8588},
 {'neg': 0.342, 'neu': 0.564, 'pos': 0.094, 'compound': -0.7639},
 {'neg': 0.165, 'neu': 0.551, 'pos': 0.284, 'compound': 0.3343}]

In [6]:
# review data
# load review data for las vegas hotels 
reviews = pd.read_csv('../data/vegas_hotel_reviews.csv')
business = pd.read_csv('../data/vegas_hotel_info.csv')[["business_id", "name"]]

In [11]:
# add sentiment scores to dataframe
reviews["scores"] = reviews["text"].apply(lambda text: vader.polarity_scores(text))

In [8]:
# extract compound score
reviews["compound"] = reviews["scores"].apply(lambda score_dict: score_dict["compound"])

In [9]:
# compare sentiment across star ratings
scoreByStar = reviews.groupby(['stars'])['compound'].mean().reset_index()
scoreByStar 


,stars,compound
0,1,-0.121242
1,2,0.283465
2,3,0.669230
3,4,0.866542
4,5,0.889663


In [10]:
# compare sentiments across hotels
scoreByHotel = reviews.groupby(['business_id'])['compound'].mean().reset_index()
scoreByHotel = scoreByHotel.merge(business, on = 'business_id').sort_values(by=['compound'], ascending = False)
scoreByHotel

,business_id,compound,name
0,-7yF42k0CcJhtPw51oaOqQ,0.754411,Bellagio
4,AtjsjFzalWqJ7S9DUFQ4bw,0.750885,The Cosmopolitan of Las Vegas
1,34uJtlPnKicSaX1V8_tu1A,0.749164,The Venetian Resort Hotel Casino
5,DUdBbrvqfaqUe9GYKmYNtA,0.725046,The Mirage
7,JpHE7yhMS5ehA9e8WG_ETg,0.704415,Aria Hotel & Casino
6,HMr_KN63f6MzM9h8Wije3Q,0.687764,New York - New York
16,uJYw4p59AKh8c8h5yWMdOw,0.655951,Planet Hollywood Las Vegas Resort & Casino
10,TWD8c5-P7w9v-2KX_GSNZQ,0.648367,Mandalay Bay Resort & Casino
11,_v6HIliEOn0l0YaAmBOrxw,0.639970,Treasure Island
13,eWPFXL1Bmu1ImtIa2Rqliw,0.629654,MGM Grand Hotel


In [12]:
# lowest scored review
reviews[reviews['compound'] == reviews['compound'].min()]['text'].values[0]

'This place RUINED our wedding. Poison, attempted rape, disaster. We should have pressed charges.  \n\nMy husband and I came to Vegas to elope when we were 19 (Late 2007). Our naivety was taken advantage of. \n\nEverything was alright until we ordered room service on our first morning which was a deadly mistake. Eight hours later my husband was beginning to feel sick. We had ordered different things at breakfast and what he had must have been bad. We hung out at the room and went to bed early hoping he\'d get better. But, I woke up in the middle of the night to find that my husband was throwing up in the bathroom. He didn\'t come back to bed. I fell in and out of sleep. In the morning I found that he had spent the night in the shower due to severe uncontrollable diarrhea. He was still throwing up and still had diarrhea. He had gotten bad food poisoning from the room service food and should have gone to the hospital, but we didn\'t have a lot of money and weren\'t sure how much an out o